In [1]:
from qiskit.optimization import QuadraticProgram

The QuadraticProgram supports three types of variables: 
- Binary type  (displayed in LP format as 'Binaries')
- Integer type (displayed in LP format as 'Generals')
- Continuous type (not displayed in LP format)

For a variable, one can specify name, type, lower bound and upper bound.

Continuous variable has default<br> 
lower bound = 0 and <br>
upper bound = infinity. 

<i>Note: We cannot use ‘e’ or ‘E’ as the first character of names due to the specification of LP format.</i>

# Method 1: Creating QuadraticProgram from DocPlex Model

In [2]:
# Step 1: Make a Docplex model
from docplex.mp.model import Model
dcpxMdl = Model('AMC Docplex Model')
x = dcpxMdl.binary_var('x')
y = dcpxMdl.integer_var(lb=-1, ub=5, name='y')
dcpxMdl.minimize(2*x + 7 * y)
dcpxMdl.add_constraint(x - y == 3)
dcpxMdl.add_constraint((x + y) * (x - y) <= 1)
print(dcpxMdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Docplex Model

Minimize
 obj: 2 x + 7 y
Subject To
 c1: x - y = 3
 qc1: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



In [3]:
#Step 2: Create the QP from the Docplex model
mod = QuadraticProgram()
mod.from_docplex(dcpxMdl)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Docplex Model

Minimize
 obj: 2 x + 7 y
Subject To
 c0: x - y = 3
 q0: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



# Method 2: Directly constructing a QuadraticProgram

In [4]:
#We start from an empty model.
mod = QuadraticProgram('AMC Direct QP')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj:
Subject To

Bounds
End



In [12]:
# Add variables
mod.binary_var(name='x')
mod.integer_var(name='y', lowerbound=-1, upperbound=5)
mod.continuous_var(name='z', lowerbound=-1, upperbound=5)
print(mod.export_as_lp_string())

QiskitOptimizationError: 'Variable name already exists: x'

Now we set the objective function by invoking QuadraticProgram.minimize or QuadraticProgram.maximize. 
We may add <br>
a) a constant term, i.e. a value, integer or continuous, e.g., constant=4.5<br>
b) linear objective function {'x': coefficient, ...} e.g., linear={'p':3.2, 'x':2} and <br>
c) quadratic objective function {('x','y'): coefficient, ...} <br>
with linear and quadratic terms be either list, matrix or dictionary.

<i>Note: In LP format the quadratic part has to be scaled by a factor 1/2.</i> 

Thus, when printing as LP format, the quadratic part is first multiplied by 2 and then divided by 2 again.



In [13]:
# Add objective function using dictionaries
mod.minimize(constant=3, linear={'x': 1}, quadratic={('x', 'y'): 2, ('z', 'z'): -1})
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



Another way to specify the quadratic program is using arrays. 

For the linear term, the array corresponds to the vector c in the mathematical formulation. 

For the quadratic term, the array corresponds to the matrix Q. 

<i>Note: Ordering of the variables (x in the mathematical formulation) is the order in which the variables were originally declared in the QuadraticProgram object.</i>

In [11]:
# Add objective function using lists/arrays
mod.minimize(linear=[1,0], quadratic=[[0,1],[1,-1]])
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: x + [ 4 x*y - 2 y^2 ]/2
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [14]:
#Access the constant, the linear term, and the quadratic term from Quadratic.objective
print('constant:\t\t\t', mod.objective.constant)
print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())

#As for linear and quadratic terms, we get a dense matrix (to_array), a sparse matrix (coefficients), 
#and a dictionary (to_dict). 
#For dictionaries, we may specify whether to use variable indices or names as keys. 
#Note that the quadratic terms are stored in a compressed way, 
#e.g., {('x', 'y'): 1, ('y', 'x'): 2} is stored as {('x', 'y'): 3}. 
#You can get the quadratic term as a symmetric matrix by calling to_array(symmetric=True) or 
#to_dict(symmetric=True). 
#If you call to_dict(name=True), you can get a dictionary whose keys are pairs of variable names.
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')
print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)

constant:			 3
linear dict:			 {0: 1.0}
linear array:			 [1. 0. 0.]
linear array as sparse matrix:
   (0, 0)	1.0 

quadratic dict w/ index:	 {(0, 1): 2.0, (2, 2): -1.0}
quadratic dict w/ name:		 {('x', 'y'): 2.0, ('z', 'z'): -1.0}
symmetric quadratic dict w/ name:	 {('y', 'x'): 1.0, ('x', 'y'): 1.0, ('z', 'z'): -1.0}
quadratic matrix:
 [[ 0.  2.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]] 

symmetric quadratic matrix:
 [[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]] 

quadratic matrix as sparse matrix:
   (0, 1)	2.0
  (2, 2)	-1.0


In [15]:
#Adding/removing linear and quadratic constraints
#You can add linear constraints by setting 
#name, linear expression, sense (‘EQ’, ‘LE’, and ‘GE’ as Docplex supports) and right-hand-side value (rhs). 


# Add linear constraints
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='==', rhs=3, name='lin_eq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='<=', rhs=3, name='lin_leq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='>=', rhs=3, name='lin_geq')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [17]:
# Add quadratic constraints
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='==', rhs=1, name='quad_eq')
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='<=', rhs=1, name='quad_leq')
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='>=', rhs=1, name='quad_geq')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3
 quad_eq: [ x^2 - y*z ] + x + y = 1
 quad_leq: [ x^2 - y*z ] + x + y <= 1
 quad_geq: [ x^2 - y*z ] + x + y >= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [18]:
#You can access linear and quadratic terms of linear and quadratic constraints as in the same way as 
#the objective function.
lin_geq = mod.get_linear_constraint('lin_geq')
print('lin_geq:', lin_geq.linear.to_dict(use_name=True), lin_geq.sense, lin_geq.rhs)
quad_geq = mod.get_quadratic_constraint('quad_geq')
print('quad_geq:', quad_geq.linear.to_dict(use_name=True), quad_geq.quadratic.to_dict(use_name=True), quad_geq.sense, lin_geq.rhs)

lin_geq: {'x': 1.0, 'y': 2.0} ConstraintSense.GE 3
quad_geq: {'x': 1.0, 'y': 1.0} {('x', 'x'): 1.0, ('y', 'z'): -1.0} ConstraintSense.GE 3


In [19]:
#You can also remove linear/quadratic constraints by remove_linear_constraint and remove_quadratic_constraint.

# Remove constraints
mod.remove_linear_constraint('lin_geq')
mod.remove_quadratic_constraint('quad_geq')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 quad_eq: [ x^2 - y*z ] + x + y = 1
 quad_leq: [ x^2 - y*z ] + x + y <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [20]:
#You can substitute some of variables with constants or other variables. 
#QuadraticProgram has a method substitute_variables(constants=..., variables=...) 
#to deal with the following two cases. 
#- x←c: when constants have a dictionary {x: c}. 
#- x←cy: when variables have a dictionary {x: (y, c)}.

#Substituting Variables
#x is replaced with 0
#y is replaced with -z
sub = mod.substitute_variables(constants={'x': 0}, variables={'y': ('z', -1)})
print(sub.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: [ - 2 z^2 ]/2 + 3
Subject To
 lin_eq: - 2 z = 3
 lin_leq: - 2 z <= 3
 quad_eq: [ z^2 ] - z = 1
 quad_leq: [ z^2 ] - z <= 1

Bounds
 -1 <= z <= 1
End



In [21]:
#If the resulting problem is infeasible due to lower bounds or upper bounds, 
#the methods returns the status Status.INFEASIBLE. 
#If we try to replace variable x with -1 - now -1 is out of range of x (0 <= x <= 1). 
#So, it returns Status.INFEASIBLE.
sub = mod.substitute_variables(constants={'x': -1})
print(sub.export_as_lp_string())
print(sub.status)

Infeasible substitution for variable: x


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Direct QP

Minimize
 obj: - 2 y + [ - 2 z^2 ]/2 + 2
Subject To
 lin_eq: 2 y = 4
 lin_leq: 2 y <= 4
 quad_eq: [ - y*z ] + y = 1
 quad_leq: [ - y*z ] + y <= 1

Bounds
 -1 <= y <= 5
 -1 <= z <= 5

Generals
 y
End

QuadraticProgramStatus.INFEASIBLE


In [22]:
#You cannot substitute variables multiple times. The method raises an error in such a case.
from qiskit.optimization import QiskitOptimizationError
try:
    sub = mod.substitute_variables(constants={'x': -1}, variables={'y': ('x', 1)})
except QiskitOptimizationError as e:
    print('Error: {}'.format(e))



Error: 'Cannot substitute by variable that gets substituted itself: y <- x 1'
